In [ ]:
import datetime
from sklearn import metrics, model_selection, ensemble
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
import xgboost as xgb
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import xgboost

In [ ]:
try: 
    import optuna
except:
    !pip install optuna
    import optuna

In [ ]:
try:
    train = pd.read_csv("./train_for_crossval.csv", delimiter=",", sep='.')
except:
    train = pd.read_csv("./sample_data/train_for_crossval.csv", delimiter=",", sep='.')

In [ ]:
train

In [4]:
X = train.drop(["target"], axis=1)
y = train["target"]

In [5]:
num_train = train.select_dtypes([int, float])
cat_train = train.select_dtypes(object)

num = list(num_train.drop(['target'],axis=1))
cat = list(cat_train)

In [6]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OrdinalEncoder

rmse = make_scorer(mean_squared_error, squared=False)

pipeline_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaling', StandardScaler()),  
    ('normal', PowerTransformer()), 
])
pipeline_cat = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
    # ('encoding', OrdinalEncoder()),
    ('encoding', OneHotEncoder(handle_unknown='ignore')),
])
preprocessor = ColumnTransformer(n_jobs=-1,
    transformers=[
        ('num', pipeline_num, num),
        ('cat', pipeline_cat, cat),
        ], remainder="passthrough")

transformed = preprocessor.fit_transform(X)

# svd = TruncatedSVD(n_components=20, n_iter=7, random_state=42)
# transformed = svd.fit_transform(transformed)

def objective(trial):
    
    param_model = {
        # 'tree_method':'gpu_hist',
        'n_estimators': 5000,
        'eval_metric':'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1),
        'max_depth': trial.suggest_int('max_depth', 4, 6),
        'objective': trial.suggest_categorical("objective", ['reg:squarederror']),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 100),
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        'subsample': trial.suggest_float('subsample', 0.5, 0.9)
    }
    param_svd = {
        'n_components': trial.suggest_categorical("n_components", [30, 50, 70]),
    }
    
    pipeline_ridge = Pipeline(steps=[
                                    #  ('preprocessor', preprocessor), 
#                                      ('svd', TruncatedSVD(random_state=42, **param_svd)), 
                                     ('model', xgboost.XGBRegressor(**param_model)),
                                     ])

    rmse_mean_cv = cross_val_score(pipeline_ridge, transformed, y, cv=5, scoring=rmse, n_jobs=-1)
    print(rmse_mean_cv)

    return rmse_mean_cv.mean()

In [ ]:
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=10)

[I 2021-08-29 10:50:14,438] A new study created in memory with name: no-name-1de0fb81-e4e2-472f-ab6e-fd236b4fe649


In [ ]:
# 73.63670041849919 с бинами
# 73.9315395633810 без бинов
# 74.38762034243373 pca + бины
# 73.61114198701696 бины+кластер
# 73.10832265456563 настраеваемые бины
# 0.7147909093792476 kaggle

In [ ]:
best_params = study.best_trial.params
best_params